In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = "0"
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ['TORCH_USE_CUDA_DSA'] = "1"

from transformers import AutoTokenizer, AutoModelForCausalLM
from lib import Datasets
from datasets import load_dataset
import copy
from transformers import Trainer, TrainingArguments
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence, List, Literal

import torch
import transformers
from transformers import Trainer
from datasets import load_dataset

2025-02-15 13:11:00.759581: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-15 13:11:00.774162: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739614260.791204 3924877 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739614260.796231 3924877 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-15 13:11:00.814914: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
IGNORE_INDEX = -100

# PROMPT = (
#         "Below is an instruction that describes a task. "
#         "Write a response that appropriately completes the request.\n\n"
#         "### Instruction:\n{instruction}\n\n### Response:"
#     )

PROMPT = (
        "### Task:\n{instruction}\n\n### Solution:"
    )

In [3]:
from peft import LoraConfig, TaskType, get_peft_model

peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1, target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'up_proj', 'down_proj'])





# name = "meta-llama/Llama-2-7b-hf"
# name = "distilgpt2"
name = "meta-llama/Llama-3.2-1B"

# tokenizer = AutoTokenizer.from_pretrained(name)
model = AutoModelForCausalLM.from_pretrained(name, torch_dtype=torch.float16, device_map='balanced')

# model = get_peft_model(model, peft_config)

print(model)


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

In [4]:
# --data_path meta-math/MetaMathQA \
# --dataset_field query response \
    
# python -u train_model.py \
#     --model_name_or_path $BASE_MODEL \
#     --output_dir $OUTPUT \
#     --corda_mode False \
#     --lora_r 128 \
#     --data_path meta-math/MetaMathQA \
#     --dataset_split "train[:100000]" \
#     --dataset_field query response \
#     --num_train_epochs 1 \
#     --per_device_train_batch_size 1 \
#     --gradient_accumulation_steps 128 \
#     --save_strategy "steps" \
#     --save_steps 100 \
#     --save_total_limit 1 \
#     --learning_rate 2e-5 \
#     --weight_decay 0. \
#     --warmup_ratio 0.03 \
#     --lr_scheduler_type "cosine" \
#     --logging_steps 1 \
#     --bf16 True \
#     --tf32 True \
#     --report_to none

In [5]:
def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            padding="longest",
            # padding='max_length',
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )


In [6]:
def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        # print(len(label), source_len)
        # label[:source_len] = IGNORE_INDEX
        a = 5
        
    return dict(input_ids=input_ids, labels=labels)

def train_tokenize_function(examples, tokenizer, query, response):
    sources = [PROMPT.format_map(dict(instruction=instruction)) for instruction in examples[query]]
    targets = [f"{output}{tokenizer.eos_token}" for output in examples[response]]
    data_dict = preprocess(sources, targets, tokenizer)
    return data_dict

In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
        name,
        model_max_length=1024,
        padding_side="right",
        use_fast=True,
        trust_remote_code=True,
    )

tokenizer.pad_token_id = tokenizer.eos_token_id

raw_train_datasets = load_dataset("meta-math/MetaMathQA", split='train[:10]')

train_dataset = raw_train_datasets.map(
        train_tokenize_function,
        batched=True,
        batch_size=3000,
        num_proc=10, # 32
        remove_columns=raw_train_datasets.column_names,
        load_from_cache_file=True,
        desc="Running tokenizer on train dataset",
        fn_kwargs={"tokenizer": tokenizer, "query": "query", "response": "response"}
    )
    

Running tokenizer on train dataset (num_proc=10):   0%|          | 0/10 [00:00<?, ? examples/s]

In [8]:
@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = [torch.tensor(x) for x in input_ids]
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = [torch.tensor(x) for x in labels]
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )


In [9]:
training_args = TrainingArguments(
    output_dir='./logs/abrakkk',          
    num_train_epochs=1,                   
    per_device_train_batch_size=1,       
    per_device_eval_batch_size=1,       
    gradient_accumulation_steps=1,
    eval_accumulation_steps=10,
    learning_rate=2e-5,               
    weight_decay=0.0,                   
    warmup_ratio=0.03,                   
    lr_scheduler_type="cosine",          
    logging_steps=1,                       
    fp16=True,
    report_to="tensorboard",              
    logging_dir='./logs/abra',                 
    # use_cpu=True,
)

In [10]:
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
data_module = dict(train_dataset=train_dataset, data_collator=data_collator)
model.config.use_cache = False

In [11]:
from transformers import Trainer, TrainingArguments, EvalPrediction
from sklearn.metrics import accuracy_score
import numpy as np
from evaluate import load


metric = load('accuracy')

import numpy as np
from evaluate import load

# Load the accuracy metric
metric = load('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    predictions = np.argmax(predictions, axis=2).flatten()
    
    labels = labels.flatten()
    
    mask = labels != -100
    
    return metric.compute(predictions=predictions[mask], references=labels[mask])


In [12]:
# trainer = Trainer(model=model, tokenizer=tokenizer, args=script_args, **data_module)

# trainer = Trainer(model=model, args=training_args, **data_module)


trainer = Trainer(model=model, args=training_args, compute_metrics=compute_metrics, **data_module)

In [13]:
print(data_module)

{'train_dataset': Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 10
}), 'data_collator': DataCollatorForSupervisedDataset(tokenizer=PreTrainedTokenizerFast(name_or_path='meta-llama/Llama-3.2-1B', vocab_size=128000, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>', 'pad_token': '<|end_of_text|>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004:

In [14]:
# from transformers import pipeline

# # pl = pipeline(
# #     "text-generation",
# #     model=model,
# #     tokenizer=tokenizer,
# #     torch_dtype=torch.float16
# # )

# # with torch.inference_mode(), torch.cuda.amp.autocast():
# #     print(pl('Hello ', max_new_tokens=16,))

# pl = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     torch_dtype=torch.float16
# )

# with torch.inference_mode(), torch.cuda.amp.autocast():
#     print(pl('Hello ', max_new_tokens=16, do_sample=False, temperature=None, top_p=None,))

In [15]:
import math
import time

# model.eval()
# model.config.use_cache = True

# trainer.train()

eval_results = trainer.evaluate(eval_dataset=trainer.train_dataset)
print(eval_results)

{'eval_loss': nan, 'eval_model_preparation_time': 0.0006, 'eval_accuracy': 0.0, 'eval_runtime': 35.5647, 'eval_samples_per_second': 0.281, 'eval_steps_per_second': 0.281}


In [16]:
print(data_module['train_dataset']['input_ids'])

[[128000, 14711, 5546, 512, 6600, 47383, 323, 13142, 527, 19301, 5219, 389, 279, 6485, 11277, 13, 13142, 41011, 279, 1486, 400, 16, 10, 17, 72, 13244, 2895, 47383, 41011, 400, 12, 16, 21905, 13244, 2650, 3117, 10980, 527, 2895, 47383, 323, 13142, 596, 3585, 1980, 14711, 12761, 75145, 6138, 1990, 1403, 3585, 5035, 87, 62, 16, 7509, 62, 16, 15437, 323, 5035, 87, 62, 17, 7509, 62, 17, 15437, 304, 279, 6485, 11277, 374, 2728, 555, 279, 15150, 59060, 27986, 97165, 87, 62, 17, 6695, 62, 16, 30876, 17, 13666, 88, 62, 17, 12303, 62, 16, 30876, 17, 32816, 627, 644, 420, 1162, 11, 13142, 596, 1486, 374, 5035, 16, 11, 17, 15437, 323, 2895, 47383, 596, 1486, 374, 400, 4172, 16, 11, 16, 15437, 627, 4516, 279, 6138, 1990, 872, 3585, 374, 59060, 27986, 90, 56034, 16, 52456, 16, 97959, 17, 10, 1209, 16, 52456, 17, 97959, 17, 92, 35533, 27986, 90, 4172, 17, 30876, 17, 10, 4172, 16, 30876, 17, 92, 35533, 27986, 90, 19, 10, 16, 92, 35533, 27986, 90, 20, 32816, 627, 55915, 11, 2895, 47383, 323, 13142, 596

In [17]:
for e in trainer.train_dataset:
    print(len(e['input_ids']))

206
146
253
93
218
316
192
149
228
104
